*   This Notebook contains the code used to generate the Benchmark tables from the Numerical results section.

*   Connect the Notebook to a GPU (e.g. T4 or A100) in order to run the code

# Environment settings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/MA2/Master Thesis/project')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

import torch
from nn_tools.nn_compression import (Model, knapsack_greedy_compression, fool_svd_compression, count_parameters, plot_rank_ratio_heatmap)
from nn_tools import algebraic as alg
from nn_tools import fine_tuning as ft

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#The number of classes corresponding to each classification dataset from the GLUE benchmark
classification_dict = {
    "cola" : 2,
    "sst2" : 2,
    "mrpc" : 2,
    "qqp" : 2,
    "mnli" : 3,
    "qnli" : 2,
    "rte" : 2,
    "wnli" : 2,
}

# Base benchmark

Evaluates the fine-tuned models on the GLUE benchmark

In [ ]:
name_ = ['bert', 'gpt2', 'roberta']
dataset_name_ = ['cola', 'mrpc', 'rte', 'qnli', 'wnli']

task = 'classification'
split = "validation"

results = []

for name in name_ :
  for dataset_name in dataset_name_ :
      print(f'\n\n name : {name}, dataset_name : {dataset_name}')
      current_result = {}

      weight_path = f'/content/drive/MyDrive/MA2/Master Thesis/project/models/{name}/{name}_{dataset_name}.pth'
      n_class = classification_dict[dataset_name]

      my_Model = Model(name, task, weight_path)
      my_Model.tokenize_dataset(dataset_name, split, train_fraction=1, test_fraction=1)

      base_acc = ft.evaluate(my_Model, dataset_name, split=split)


      current_result['model'] = name
      current_result['dataset'] = dataset_name
      current_result['base accuracy'] = base_acc
      current_result['benchmark'] = ft.evaluate_glue_task(my_Model.model, my_Model.tokenizer, dataset_name, batch_size=32, device=my_Model.device)

      results.append(current_result)



 name : bert, dataset_name : cola


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...



 name : roberta, dataset_name : wnli


Map:   0%|          | 0/71 [00:00<?, ? examples/s]

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

eval accuracy: 0.5633802816901409


Map:   0%|          | 0/71 [00:00<?, ? examples/s]

In [ ]:
print(results)
with open("/content/drive/MyDrive/MA2/Master Thesis/project/results/benchmark/base_benchmark.pkl", 'wb') as f:
    pickle.dump(results, f)

[{'model': 'bert', 'dataset': 'cola', 'base accuracy': 0.8245445829338447, 'benchmark': {'matthews': np.float64(0.5711485966856213)}}, {'model': 'bert', 'dataset': 'mrpc', 'base accuracy': 0.8480392156862745, 'benchmark': {'accuracy': 0.8406862745098039, 'f1': 0.8888888888888888}}, {'model': 'bert', 'dataset': 'rte', 'base accuracy': 0.6389891696750902, 'benchmark': {'accuracy': 0.631768953068592}}, {'model': 'bert', 'dataset': 'qnli', 'base accuracy': 0.8663737872963573, 'benchmark': {'accuracy': 0.8960278235401794}}, {'model': 'bert', 'dataset': 'wnli', 'base accuracy': 0.5633802816901409, 'benchmark': {'accuracy': 0.2676056338028169}}, {'model': 'gpt2', 'dataset': 'cola', 'base accuracy': 0.7785234899328859, 'benchmark': {'matthews': np.float64(0.43825163224076547)}}, {'model': 'gpt2', 'dataset': 'mrpc', 'base accuracy': 0.803921568627451, 'benchmark': {'accuracy': 0.803921568627451, 'f1': 0.868421052631579}}, {'model': 'gpt2', 'dataset': 'rte', 'base accuracy': 0.631768953068592, '

# Fool benchmark

Compression of the fine-tuned models with SVD, using a uniform truncation rank

In [ ]:
name_ = ['bert', 'gpt2', 'roberta']
dataset_name_ = ['cola', 'mrpc', 'rte', 'qnli', 'wnli']
rank_ratio_ = [0.6, 0.5, 0.1]

task = 'classification'
split = "validation"

results = []

for name in name_ :
  for dataset_name in dataset_name_ :
    for rank_ratio in rank_ratio_ :
      print(f'\n\n name : {name}, dataset_name : {dataset_name}, rank ratio : {rank_ratio}')
      current_result = {}

      weight_path = f'/content/drive/MyDrive/MA2/Master Thesis/project/models/{name}/{name}_{dataset_name}.pth'
      n_class = classification_dict[dataset_name]

      my_Model = Model(name, task, weight_path)
      my_Model.tokenize_dataset(dataset_name, split, train_fraction=1, test_fraction=1)

      best_Model = fool_svd_compression(my_Model, rank_ratio=rank_ratio)
      res_param = count_parameters(best_Model.model) / count_parameters(my_Model.model)
      pre_acc = ft.evaluate(best_Model, dataset_name, split=split)
      print(f'resulting params : {res_param}')


      tuned_Model = ft.fine_tune(best_Model, dataset_name, train_fraction=1, test_fraction=1, n_epochs=3, batch_size=16)
      post_acc = ft.evaluate(tuned_Model, dataset_name, split=split)

      current_result['model'] = name
      current_result['dataset'] = dataset_name
      current_result['rank ratio'] = rank_ratio
      current_result['resulting parameters'] = res_param
      current_result['pre accuracy'] = pre_acc
      current_result['post accuracy'] = post_acc
      current_result['pre benchmark'] = ft.evaluate_glue_task(best_Model.model, best_Model.tokenizer, dataset_name, batch_size=32, device=best_Model.device)
      current_result['post benchmark'] = ft.evaluate_glue_task(tuned_Model.model, tuned_Model.tokenizer, dataset_name, batch_size=32, device=tuned_Model.device)

      results.append(current_result)



 name : bert, dataset_name : cola, rank ratio : 0.6
computing SVD of the model's layers... 



eval accuracy: 0.4074784276126558
resulting params : 0.9212104646224393


Step,Training Loss
500,0.339900
1000,0.230800
1500,0.145800


eval accuracy: 0.8216682646212847


 name : bert, dataset_name : cola, rank ratio : 0.5
computing SVD of the model's layers... 



eval accuracy: 0.32694151486097794
resulting params : 0.8060565283013891


...

Step,Training Loss


eval accuracy: 0.43661971830985913


 name : roberta, dataset_name : wnli, rank ratio : 0.1
computing SVD of the model's layers... 



eval accuracy: 0.43661971830985913
resulting params : 0.41974480447490303


Step,Training Loss


eval accuracy: 0.5633802816901409


In [ ]:
print(results)
with open("/content/drive/MyDrive/MA2/Master Thesis/project/results/benchmark/fool_compression.pkl", 'wb') as f:
    pickle.dump(results, f)

[{'model': 'bert', 'dataset': 'cola', 'rank ratio': 0.6, 'resulting parameters': 0.9212104646224393, 'pre accuracy': 0.4074784276126558, 'post accuracy': 0.8216682646212847, 'pre benchmark': {'matthews': np.float64(0.5606238208505129)}, 'post benchmark': {'matthews': np.float64(0.5606238208505129)}}, {'model': 'bert', 'dataset': 'cola', 'rank ratio': 0.5, 'resulting parameters': 0.8060565283013891, 'pre accuracy': 0.32694151486097794, 'post accuracy': 0.8111217641418984, 'pre benchmark': {'matthews': np.float64(0.5332198659134496)}, 'post benchmark': {'matthews': np.float64(0.5332198659134496)}}, {'model': 'bert', 'dataset': 'cola', 'rank ratio': 0.1, 'resulting parameters': 0.3393800495266066, 'pre accuracy': 0.3087248322147651, 'post accuracy': 0.6912751677852349, 'pre benchmark': {'matthews': 0.0}, 'post benchmark': {'matthews': 0.0}}, {'model': 'bert', 'dataset': 'mrpc', 'rank ratio': 0.6, 'resulting parameters': 0.9212104646224393, 'pre accuracy': 0.7573529411764706, 'post accurac

# Compression without post fine-tuning / Task-specific proxy

Compression of fine-tuned models, using a task-specific proxy and without post fine-tuning

In [ ]:
name_ = ['bert', 'gpt2', 'roberta']
dataset_name_ = ['cola', 'mrpc', 'rte', 'qnli', 'wnli']
tau_ = [0.25, 0.5]

task = 'classification'
split = "validation"
alpha = 0.1
method = 'afm'
error_type = 'distillation'
lr_metric = 'effective_rank'
proxy_type = 'dataset'
complexity_metric = 'max_param'

results = []

for name in name_ :
  for dataset_name in dataset_name_ :
    for tau in tau_ :
      print(f'\n\n name : {name}, dataset_name : {dataset_name}, tau : {tau}')
      current_result = {}

      weight_path = f'/content/drive/MyDrive/MA2/Master Thesis/project/models/{name}/{name}_{dataset_name}.pth'
      n_class = classification_dict[dataset_name]

      my_Model = Model(name, task, weight_path)
      my_Model.tokenize_dataset(dataset_name, split, train_fraction=1, test_fraction=1)

      base_acc = ft.evaluate(my_Model, dataset_name, split=split)

      best_Model, best_rank_ratio_dict, res_param = knapsack_greedy_compression(my_Model,
                                                                    min_rank_ratios=np.linspace(0.1, 0.6, 6), max_rank_ratio=0.7, rank_ratio_step=0.1, output_threshold=tau,
                                                                    proxy_type = proxy_type, dataset_name = dataset_name, batch=16, seq_len=10, hidden_size=768, mu=0, sigma=1,
                                                                    complexity_metric = complexity_metric, lr_metric = lr_metric, error_type=error_type, method=method, base_accuracy = base_acc, alpha=alpha
                                                                    )
      acc = ft.evaluate(best_Model, dataset_name, split=split)
      current_result['model'] = name
      current_result['dataset'] = dataset_name
      current_result['tau'] = tau
      current_result['alpha'] = alpha
      current_result['method'] = method
      current_result['resulting parameters'] = res_param
      current_result['relative accuracy'] = acc/base_acc
      current_result['benchmark'] = ft.evaluate_glue_task(best_Model.model, best_Model.tokenizer, dataset_name, batch_size=32, device=best_Model.device)

      results.append(current_result)



 name : bert, dataset_name : cola, tau : 0.25


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

eval accuracy: 0.5633802816901409


Map:   0%|          | 0/71 [00:00<?, ? examples/s]



 name : roberta, dataset_name : wnli, tau : 0.5


eval accuracy: 0.5633802816901409


eval accuracy: 0.5492957746478874


In [ ]:
print(results)
with open("/content/drive/MyDrive/MA2/Master Thesis/project/results/benchmark/no_post_fine_tuning__task_specific.pkl", 'wb') as f:
    pickle.dump(results, f)

[{'model': 'bert', 'dataset': 'cola', 'tau': 0.25, 'alpha': 0.1, 'method': 'afm', 'resulting parameters': 0.9245396427587655, 'relative accuracy': 0.9709302325581395, 'benchmark': {'matthews': np.float64(0.5484845164994472)}}, {'model': 'bert', 'dataset': 'cola', 'tau': 0.5, 'alpha': 0.1, 'method': 'afm', 'resulting parameters': 0.8705205167472391, 'relative accuracy': 0.8895348837209301, 'benchmark': {'matthews': np.float64(0.4681762291184293)}}, {'model': 'bert', 'dataset': 'mrpc', 'tau': 0.25, 'alpha': 0.1, 'method': 'afm', 'resulting parameters': 0.9244863197906816, 'relative accuracy': 0.7601156069364162, 'benchmark': {'accuracy': 0.8259803921568627, 'f1': 0.8834154351395731}}, {'model': 'bert', 'dataset': 'mrpc', 'tau': 0.5, 'alpha': 0.1, 'method': 'afm', 'resulting parameters': 0.8867021194683289, 'relative accuracy': 0.5346820809248555, 'benchmark': {'accuracy': 0.7965686274509803, 'f1': 0.865040650406504}}, {'model': 'bert', 'dataset': 'rte', 'tau': 0.25, 'alpha': 0.1, 'method

# Compression without post fine-tuning / Random proxy

Compression of fine-tuned models, using a random proxy and without post fine-tuning

In [ ]:
name_ = ['bert', 'gpt2', 'roberta']
dataset_name_ = ['cola', 'mrpc', 'rte', 'qnli', 'wnli']
tau_ = [0.1, 0.25, 0.5]

task = 'classification'
split = "validation"
alpha = None
method = 'svd'
error_type = 'hidden_output'
lr_metric = 'effective_rank'
proxy_type = 'random'
complexity_metric = 'n_param'

results = []

for name in name_ :
  for dataset_name in dataset_name_ :
    for tau in tau_ :
      print(f'\n\n name : {name}, dataset_name : {dataset_name}, tau : {tau}')
      current_result = {}

      weight_path = f'/content/drive/MyDrive/MA2/Master Thesis/project/models/{name}/{name}_{dataset_name}.pth'
      n_class = classification_dict[dataset_name]

      my_Model = Model(name, task, weight_path)
      my_Model.tokenize_dataset(dataset_name, split, train_fraction=1, test_fraction=1)

      base_acc = ft.evaluate(my_Model, dataset_name, split=split)

      best_Model, best_rank_ratio_dict, res_param = knapsack_greedy_compression(my_Model,
                                                                    min_rank_ratios=np.linspace(0.1, 0.6, 6), max_rank_ratio=0.7, rank_ratio_step=0.1, output_threshold=tau,
                                                                    proxy_type = proxy_type, dataset_name = dataset_name, batch=16, seq_len=10, hidden_size=768, mu=0, sigma=1,
                                                                    complexity_metric = complexity_metric, lr_metric = lr_metric, error_type=error_type, method=method, base_accuracy = base_acc, alpha=alpha
                                                                    )
      acc = ft.evaluate(best_Model, dataset_name, split=split)
      current_result['model'] = name
      current_result['dataset'] = dataset_name
      current_result['tau'] = tau
      current_result['alpha'] = alpha
      current_result['method'] = method
      current_result['resulting parameters'] = res_param
      current_result['relative accuracy'] = acc/base_acc
      current_result['benchmark'] = ft.evaluate_glue_task(best_Model.model, best_Model.tokenizer, dataset_name, batch_size=32, device=best_Model.device)

      results.append(current_result)



 name : bert, dataset_name : cola, tau : 0.1


eval accuracy: 0.8245445829338447


eval accuracy: 0.8245445829338447


 name : bert, dataset_name : cola, tau : 0.25


eval accuracy: 0.8245445829338447


eval accuracy: 0.825503355704698


 name : bert, dataset_name : cola, tau : 0.5


...

eval accuracy: 0.5352112676056338


 name : roberta, dataset_name : wnli, tau : 0.25


eval accuracy: 0.5633802816901409


eval accuracy: 0.4225352112676056


 name : roberta, dataset_name : wnli, tau : 0.5


eval accuracy: 0.5633802816901409


eval accuracy: 0.43661971830985913


In [ ]:
print(results)
with open("/content/drive/MyDrive/MA2/Master Thesis/project/results/benchmark/no_post_fine_tuning__random.pkl", 'wb') as f:
    pickle.dump(results, f)

[{'model': 'bert', 'dataset': 'cola', 'tau': 0.1, 'alpha': None, 'method': 'svd', 'resulting parameters': 0.9972852964573437, 'relative accuracy': 1.0, 'benchmark': {'matthews': np.float64(0.5711485966856213)}}, {'model': 'bert', 'dataset': 'cola', 'tau': 0.25, 'alpha': None, 'method': 'svd', 'resulting parameters': 0.9865176738785905, 'relative accuracy': 1.0011627906976743, 'benchmark': {'matthews': np.float64(0.573727765379499)}}, {'model': 'bert', 'dataset': 'cola', 'tau': 0.5, 'alpha': None, 'method': 'svd', 'resulting parameters': 0.9595389921600989, 'relative accuracy': 1.0046511627906975, 'benchmark': {'matthews': np.float64(0.5814477644432363)}}, {'model': 'bert', 'dataset': 'mrpc', 'tau': 0.1, 'alpha': None, 'method': 'svd', 'resulting parameters': 1, 'relative accuracy': 1.0, 'benchmark': {'accuracy': 0.8406862745098039, 'f1': 0.8888888888888888}}, {'model': 'bert', 'dataset': 'mrpc', 'tau': 0.25, 'alpha': None, 'method': 'svd', 'resulting parameters': 0.9593636237141908, 'r

# Compression with post fine-tuning / Task-specific proxy

Compression of fine-tuned models, using a task-specific proxy and with post fine-tuning for 3 epochs

In [ ]:
name_ = ['bert', 'gpt2', 'roberta']
dataset_name_ = ['cola', 'mrpc', 'rte', 'qnli', 'wnli']
tau_ = [0.5, 0.75, 1.0, 1.25]

task = 'classification'
split = "validation"
alpha = None
method = 'afm'
error_type = 'hidden_output'
lr_metric = 'effective_rank'
proxy_type = 'dataset'
complexity_metric = 'n_param'
min_rank_ratios = [0.05, 0.1, 0.2, 0.3, 0.4]

results = []

for name in name_ :
  for dataset_name in dataset_name_ :
    for tau in tau_ :
      print(f'\n\n name : {name}, dataset_name : {dataset_name}, tau : {tau}')
      current_result = {}

      weight_path = f'/content/drive/MyDrive/MA2/Master Thesis/project/models/{name}/{name}_{dataset_name}.pth'
      n_class = classification_dict[dataset_name]

      my_Model = Model(name, task, weight_path)
      my_Model.tokenize_dataset(dataset_name, split, train_fraction=1, test_fraction=1)

      base_acc = ft.evaluate(my_Model, dataset_name, split=split)

      best_Model, best_rank_ratio_dict, res_param = knapsack_greedy_compression(my_Model,
                                                                    min_rank_ratios=min_rank_ratios, max_rank_ratio=0.7, rank_ratio_step=0.1, output_threshold=tau,
                                                                    proxy_type = proxy_type, dataset_name = dataset_name, batch=16, seq_len=10, hidden_size=768, mu=0, sigma=1,
                                                                    complexity_metric = complexity_metric, lr_metric = lr_metric, error_type=error_type, method=method, base_accuracy = base_acc, alpha=alpha
                                                                    )

      tuned_Model = ft.fine_tune(best_Model, dataset_name, train_fraction=1, test_fraction=1, n_epochs=3, batch_size=16)
      acc = ft.evaluate(tuned_Model, dataset_name, split=split)

      current_result['model'] = name
      current_result['dataset'] = dataset_name
      current_result['tau'] = tau
      current_result['alpha'] = alpha
      current_result['method'] = method
      current_result['resulting parameters'] = res_param
      current_result['relative accuracy'] = acc/base_acc
      current_result['benchmark'] = ft.evaluate_glue_task(tuned_Model.model, tuned_Model.tokenizer, dataset_name, batch_size=32, device=tuned_Model.device)

      results.append(current_result)



 name : bert, dataset_name : cola, tau : 0.5


eval accuracy: 0.8245445829338447


Step,Training Loss
500,0.301600
1000,0.206600
1500,0.123700


eval accuracy: 0.8149568552253116


 name : bert, dataset_name : cola, tau : 0.75


eval accuracy: 0.8245445829338447


...

Step,Training Loss


eval accuracy: 0.5633802816901409


 name : roberta, dataset_name : wnli, tau : 1.25


eval accuracy: 0.5633802816901409


Step,Training Loss


eval accuracy: 0.5633802816901409


In [ ]:
print(results)
with open("/content/drive/MyDrive/MA2/Master Thesis/project/results/benchmark/post_fine_tuning__task_specific.pkl", 'wb') as f:
    pickle.dump(results, f)

[{'model': 'bert', 'dataset': 'cola', 'tau': 0.5, 'alpha': None, 'method': 'afm', 'resulting parameters': 0.8841059905696714, 'relative accuracy': 0.9883720930232558, 'benchmark': {'matthews': np.float64(0.5445498216235399)}}, {'model': 'bert', 'dataset': 'cola', 'tau': 0.75, 'alpha': None, 'method': 'afm', 'resulting parameters': 0.865144423496237, 'relative accuracy': 0.9732558139534884, 'benchmark': {'matthews': np.float64(0.5168494611799795)}}, {'model': 'bert', 'dataset': 'cola', 'tau': 1.0, 'alpha': None, 'method': 'afm', 'resulting parameters': 0.8463582705375768, 'relative accuracy': 0.9848837209302326, 'benchmark': {'matthews': np.float64(0.5391508045334119)}}, {'model': 'bert', 'dataset': 'cola', 'tau': 1.25, 'alpha': None, 'method': 'afm', 'resulting parameters': 0.8192743312164474, 'relative accuracy': 0.9627906976744186, 'benchmark': {'matthews': np.float64(0.49038894064526567)}}, {'model': 'bert', 'dataset': 'mrpc', 'tau': 0.5, 'alpha': None, 'method': 'afm', 'resulting p

# Compression with post fine-tuning / Random proxy

Compression of fine-tuned models, using a random proxy and with post fine-tuning for 3 epochs

In [ ]:
name_ = ['bert', 'gpt2', 'roberta']
dataset_name_ = ['cola', 'mrpc', 'rte', 'qnli', 'wnli']
tau_ = [0.5, 0.75, 1.0]

task = 'classification'
split = "validation"
alpha = None
method = 'svd'
error_type = 'hidden_output'
lr_metric = 'effective_rank'
proxy_type = 'random'
complexity_metric = 'n_param'

results = []

for name in name_ :
  for dataset_name in dataset_name_ :
    for tau in tau_ :
      print(f'\n\n name : {name}, dataset_name : {dataset_name}, tau : {tau}')
      current_result = {}

      weight_path = f'/content/drive/MyDrive/MA2/Master Thesis/project/models/{name}/{name}_{dataset_name}.pth'
      n_class = classification_dict[dataset_name]

      my_Model = Model(name, task, weight_path)
      my_Model.tokenize_dataset(dataset_name, split, train_fraction=1, test_fraction=1)

      base_acc = ft.evaluate(my_Model, dataset_name, split=split)

      best_Model, best_rank_ratio_dict, res_param = knapsack_greedy_compression(my_Model,
                                                                    min_rank_ratios=np.linspace(0.1, 0.6, 6), max_rank_ratio=0.7, rank_ratio_step=0.1, output_threshold=tau,
                                                                    proxy_type = proxy_type, dataset_name = dataset_name, batch=16, seq_len=10, hidden_size=768, mu=0, sigma=1,
                                                                    complexity_metric = complexity_metric, lr_metric = lr_metric, error_type=error_type, method=method, base_accuracy = base_acc, alpha=alpha
                                                                    )

      tuned_Model = ft.fine_tune(best_Model, dataset_name, train_fraction=1, test_fraction=1, n_epochs=3, batch_size=16)
      acc = ft.evaluate(tuned_Model, dataset_name, split=split)

      current_result['model'] = name
      current_result['dataset'] = dataset_name
      current_result['tau'] = tau
      current_result['alpha'] = alpha
      current_result['method'] = method
      current_result['resulting parameters'] = res_param
      current_result['relative accuracy'] = acc/base_acc
      current_result['benchmark'] = ft.evaluate_glue_task(tuned_Model.model, tuned_Model.tokenizer, dataset_name, batch_size=32, device=tuned_Model.device)

      results.append(current_result)



 name : bert, dataset_name : cola, tau : 0.5


eval accuracy: 0.8245445829338447


Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Step,Training Loss
500,0.213600
1000,0.147100
1500,0.101300


...

Step,Training Loss


eval accuracy: 0.5633802816901409


 name : roberta, dataset_name : wnli, tau : 1.0


eval accuracy: 0.5633802816901409


Step,Training Loss


eval accuracy: 0.5633802816901409


In [ ]:
print(results)
with open("/content/drive/MyDrive/MA2/Master Thesis/project/results/benchmark/post_fine_tuning__random.pkl", 'wb') as f:
    pickle.dump(results, f)

[{'model': 'bert', 'dataset': 'cola', 'tau': 0.5, 'alpha': None, 'method': 'svd', 'resulting parameters': 0.9595389921600989, 'relative accuracy': 0.9988372093023256, 'benchmark': {'matthews': np.float64(0.5657894359264041)}}, {'model': 'bert', 'dataset': 'cola', 'tau': 0.75, 'alpha': None, 'method': 'svd', 'resulting parameters': 0.8975637833762002, 'relative accuracy': 0.9883720930232558, 'benchmark': {'matthews': np.float64(0.5448598482839426)}}, {'model': 'bert', 'dataset': 'cola', 'tau': 1.0, 'alpha': None, 'method': 'svd', 'resulting parameters': 0.6909657246208657, 'relative accuracy': 0.9546511627906976, 'benchmark': {'matthews': np.float64(0.4734123999422081)}}, {'model': 'bert', 'dataset': 'mrpc', 'tau': 0.5, 'alpha': None, 'method': 'svd', 'resulting parameters': 0.8328458120983001, 'relative accuracy': 0.9017341040462428, 'benchmark': {'accuracy': 0.7549019607843137, 'f1': 0.8422712933753943}}, {'model': 'bert', 'dataset': 'mrpc', 'tau': 0.75, 'alpha': None, 'method': 'svd'